In [40]:
import json
import time
import numpy as np
import trimesh

# Global constants and mappings (as before)
SBE_COLORS = {
    # Walls: a warm off-white with subtle cream tones.
    "walls": [0.8, 0.8, 0.8],
    # Interior walls: almost as light as exterior, with a slightly softer tone.
    "interior_walls": [0.99, 0.98, 0.97],
    # Roofs: even lighter than walls for a very gentle contrast.
    "roofs": [0.2, 0.2, 0.2],
    # Ceilings: crisp pure white.
    "ceilings": [1.0, 1.0, 1.0],
    # Exterior floors: a light, cool concrete tone.
    "exterior_floors": [0.90, 0.88, 0.85],
    # Interior floors: a warm, soft wood-inspired tone.
    "interior_floors": [0.95, 0.92, 0.90],
    # Air walls: nearly pure white to emphasize the envelope.
    "air_walls": [0.99, 0.99, 0.99],
    # Apertures (windows): a pastel blueish-grey for a modern, airy feel.
    "apertures": [0.7, 0.75, 0.92],
    # Interior apertures (skylights, etc.): a slightly lighter version.
    "interior_apertures": [0.85, 0.88, 0.92],
    # Doors: kept dark as accent details.
    "doors": [0.20, 0.18, 0.16],
    # Interior doors: a soft dark gray.
    "interior_doors": [0.30, 0.28, 0.26],
    # Outdoor shades: a light neutral gray.
    "outdoor_shades": [0.65, 0.65, 0.65],
    # Indoor shades: a very light, airy gray.
    "indoor_shades": [0.75, 0.75, 0.75],
    # Shade meshes: a balanced mid-gray, light and unobtrusive.
    "shade_meshes": [0.55, 0.55, 0.55],
}
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Default; updated later based on boundary conditions
}
APERTURE_OFFSET = 0.1

def get_rotation_matrix(rotate = False):
    """Return a rotation matrix (rotate -90° about X-axis)."""
    if rotate:
        return trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
    else:
        return np.eye(4)

def compute_faces(num_vertices):
    """Return triangle fan faces for a polygon with num_vertices."""
    return np.array([[0, i, i + 1] for i in range(1, num_vertices - 1)], dtype=np.int64)

# --- New vectorized processing functions ---
def process_aperture_vectorized(aperture, face_type, rotation_matrix):
    """
    Process an aperture in a vectorized way:
      - Rotate its vertices
      - Compute its normal (from the first triangle)
      - Offset all vertices along that normal
      - Compute local faces for the aperture polygon
    Returns (vertices, faces, color, category) or (None, None, None, None) if invalid.
    """
    aperture_geometry = aperture.get("geometry", {})
    boundary = aperture_geometry.get("boundary", [])
    if len(boundary) < 3:
        return None, None, None, None

    vertices = np.array(boundary, dtype=np.float32)
    # Rotate all vertices at once.
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    # Compute the normal from the first triangle (vectorized)
    tri = rotated[0:3].reshape(1, 3, 3)
    normal = trimesh.triangles.normals(tri)[0]
    normal = trimesh.unitize(normal)
    # Offset all vertices along the normal.
    rotated += APERTURE_OFFSET * normal

    # Determine category and color
    if face_type == "Wall":
        category = "apertures"
    elif face_type == "RoofCeiling":
        category = "interior_apertures"
    else:
        category = "apertures"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    return rotated, local_faces, color, category

def process_face_vectorized(face, rotation_matrix):
    """
    Process a face by:
      - Rotating its vertices in one batch,
      - Computing local faces,
      - Determining its category,
      - And processing its apertures in the same vectorized fashion.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    results = {cat: [] for cat in SBE_COLORS.keys()}
    geometry = face.get("geometry", {})
    boundary = geometry.get("boundary", [])
    if len(boundary) < 3:
        return results

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))

    face_type = face.get("face_type", "Wall")
    category = FACE_TYPE_MAPPING.get(face_type, "walls")
    bc = face.get("boundary_condition", {}).get("type", "")
    if face_type == "Floor":
        category = "exterior_floors" if bc == "Ground" else "interior_floors"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    results[category].append((rotated, local_faces, color))

    # Process apertures attached to the face.
    for aperture in face.get("apertures", []):
        ap_vertices, ap_faces, ap_color, ap_cat = process_aperture_vectorized(aperture, face_type, rotation_matrix)
        if ap_vertices is not None:
            results[ap_cat].append((ap_vertices, ap_faces, ap_color))
    return results

def process_room_vectorized(room, rotation_matrix):
    """
    Process all faces in a room, returning a dictionary mapping category names
    to lists of (vertices, faces, color) tuples.
    """
    room_results = {cat: [] for cat in SBE_COLORS.keys()}
    for face in room.get("faces", []):
        face_dict = process_face_vectorized(face, rotation_matrix)
        for cat, items in face_dict.items():
            room_results[cat].extend(items)
    return room_results

def combine_category_geometry(items):
    """
    Given a list of items (each a tuple of (vertices, faces, color)) for one category,
    combine them into one large mesh using vectorized concatenation.
    Assumes all items in the list share the same vertex color.
    Returns a trimesh.Trimesh object.
    """
    if not items:
        return None

    all_vertices = []
    all_faces = []
    offset = 0
    for vertices, faces, _ in items:
        all_vertices.append(vertices)
        all_faces.append(faces + offset)
        offset += len(vertices)
    vertices_combined = np.vstack(all_vertices)
    faces_combined = np.vstack(all_faces)
    mesh = trimesh.Trimesh(vertices=vertices_combined, faces=faces_combined, process=False)
    # Use the color from the first item (they are assumed identical for a given category).
    mesh.visual.vertex_colors = items[0][2]
    return mesh


# --- Configuration & Data Loading ---
path_to_json = '../public/urban_district.hbjson'
output_path = path_to_json.replace(".hbjson", ".glb")
rotation_matrix = get_rotation_matrix()

with open(path_to_json, 'r') as f:
    hbjson = json.load(f)
rooms = hbjson.get("rooms", [])

# Instead of creating many small meshes, we accumulate geometry per category.
combined_results = {cat: [] for cat in SBE_COLORS.keys()}

t0 = time.perf_counter()
for room in rooms:
    room_geom = process_room_vectorized(room, rotation_matrix)
    for cat, items in room_geom.items():
        combined_results[cat].extend(items)
t1 = time.perf_counter()
print(f"Accumulated geometry from all rooms in {t1 - t0:.4f} seconds")

# Build one big mesh per category by concatenating vertices and faces.
meshes = []
for cat, items in combined_results.items():
    mesh = combine_category_geometry(items)
    if mesh is not None:
        meshes.append(mesh)

# Finally, combine all category meshes into one mesh.
if meshes:
    final_mesh = trimesh.util.concatenate(meshes)
    t2 = time.perf_counter()
    final_mesh.export(output_path)
    t3 = time.perf_counter()
    print(f"Exported GLB file: {output_path}")
    print(f"Mesh concatenation took {t2 - t1:.4f} seconds, export took {t3 - t2:.4f} seconds")
else:
    print("No valid geometry found in HBJSON.")




Accumulated geometry from all rooms in 1.4580 seconds
Exported GLB file: ../public/urban_district.glb
Mesh concatenation took 0.0999 seconds, export took 0.0222 seconds


In [46]:
import json
import time
import numpy as np
import trimesh

# Global constants and mappings (as before)
SBE_COLORS = {
    # Walls: a warm off-white with subtle cream tones.
    "walls": [0.8, 0.8, 0.8],
    # Interior walls: almost as light as exterior, with a slightly softer tone.
    "interior_walls": [0.99, 0.98, 0.97],
    # Roofs: using a darker tone here for contrast.
    "roofs": [0.2, 0.2, 0.2],
    # Ceilings: crisp pure white.
    "ceilings": [1.0, 1.0, 1.0],
    # Exterior floors: a light, cool concrete tone.
    "exterior_floors": [0.90, 0.88, 0.85],
    # Interior floors: a warm, soft wood-inspired tone.
    "interior_floors": [0.95, 0.92, 0.90],
    # Air walls: nearly pure white.
    "air_walls": [0.99, 0.99, 0.99],
    # Apertures (windows): a pastel blueish-grey.
    "apertures": [0.7, 0.75, 0.92],
    # Interior apertures (skylights): a slightly lighter version.
    "interior_apertures": [0.85, 0.88, 0.92],
    # Doors: kept dark as accent details.
    "doors": [0.20, 0.18, 0.16],
    # Interior doors: a soft dark gray.
    "interior_doors": [0.30, 0.28, 0.26],
    # Outdoor shades: a light neutral gray.
    "outdoor_shades": [0.65, 0.65, 0.65],
    # Indoor shades: a very light, airy gray.
    "indoor_shades": [0.75, 0.75, 0.75],
    # Shade meshes: a balanced mid-gray.
    "shade_meshes": [0.55, 0.55, 0.55],
}
FACE_TYPE_MAPPING = {
    "Wall": "walls",
    "RoofCeiling": "roofs",
    "Floor": "interior_floors",  # Default; updated later based on boundary conditions
}
APERTURE_OFFSET = 0.1

def get_rotation_matrix(rotate=False):
    """Return a rotation matrix. If 'rotate' is True, rotate -90° about X-axis; otherwise, return identity."""
    if rotate:
        return trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
    else:
        return np.eye(4)

def compute_faces(num_vertices):
    """Return triangle fan faces for a polygon with num_vertices."""
    return np.array([[0, i, i + 1] for i in range(1, num_vertices - 1)], dtype=np.int64)

# --- Vectorized processing functions ---
def process_aperture_vectorized(aperture, face_type, rotation_matrix):
    """
    Process an aperture:
      - Rotate its vertices,
      - Compute its normal (from the first triangle),
      - Offset all vertices along that normal,
      - Compute local faces.
    Returns (vertices, faces, color, category) or (None, None, None, None) if invalid.
    """
    aperture_geometry = aperture.get("geometry", {})
    boundary = aperture_geometry.get("boundary", [])
    if len(boundary) < 3:
        return None, None, None, None

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    tri = rotated[0:3].reshape(1, 3, 3)
    normal = trimesh.triangles.normals(tri)[0]
    normal = trimesh.unitize(normal)
    rotated += APERTURE_OFFSET * normal

    if face_type == "Wall":
        category = "apertures"
    elif face_type == "RoofCeiling":
        category = "interior_apertures"
    else:
        category = "apertures"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    return rotated, local_faces, color, category

def process_face_vectorized(face, rotation_matrix):
    """
    Process a face:
      - Rotate its vertices,
      - Compute local faces,
      - Determine its category,
      - Process any attached apertures.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    results = {cat: [] for cat in SBE_COLORS.keys()}
    geometry = face.get("geometry", {})
    boundary = geometry.get("boundary", [])
    if len(boundary) < 3:
        return results

    vertices = np.array(boundary, dtype=np.float32)
    rotated = trimesh.transformations.transform_points(vertices, rotation_matrix)
    local_faces = compute_faces(len(rotated))
    face_type = face.get("face_type", "Wall")
    category = FACE_TYPE_MAPPING.get(face_type, "walls")
    bc = face.get("boundary_condition", {}).get("type", "")
    if face_type == "Floor":
        category = "exterior_floors" if bc == "Ground" else "interior_floors"
    color = SBE_COLORS.get(category, [1.0, 1.0, 1.0])
    results[category].append((rotated, local_faces, color))

    for aperture in face.get("apertures", []):
        ap_vertices, ap_faces, ap_color, ap_cat = process_aperture_vectorized(aperture, face_type, rotation_matrix)
        if ap_vertices is not None:
            results[ap_cat].append((ap_vertices, ap_faces, ap_color))
    return results

def process_room_vectorized(room, rotation_matrix):
    """
    Process all faces in a room.
    Returns a dictionary mapping category names to lists of (vertices, faces, color) tuples.
    """
    room_results = {cat: [] for cat in SBE_COLORS.keys()}
    for face in room.get("faces", []):
        face_dict = process_face_vectorized(face, rotation_matrix)
        for cat, items in face_dict.items():
            room_results[cat].extend(items)
    return room_results

def combine_category_geometry(items):
    """
    Given a list of items (each a tuple of (vertices, faces, color)) for one category,
    combine them into one mesh by concatenating vertices and faces.
    Returns a trimesh.Trimesh object.
    """
    if not items:
        return None

    all_vertices = []
    all_faces = []
    offset = 0
    for vertices, faces, _ in items:
        all_vertices.append(vertices)
        all_faces.append(faces + offset)
        offset += len(vertices)
    vertices_combined = np.vstack(all_vertices)
    faces_combined = np.vstack(all_faces)
    mesh = trimesh.Trimesh(vertices=vertices_combined, faces=faces_combined, process=False)
    # Set vertex colors for debugging or fallback; this can later be overridden by material assignment.
    mesh.visual.vertex_colors = items[0][2]
    return mesh

# --- Build Scene Graph with Separate Mesh Nodes ---
def build_scene_from_category_meshes(combined_results):
    """
    For each category, combine its items into one mesh.
    Then, add each category mesh as a separate node to a scene.
    Returns a trimesh.Scene.
    """
    scene = trimesh.Scene()
    for cat, items in combined_results.items():
        mesh = combine_category_geometry(items)
        if mesh is not None:
            # Set the node name to the category so that you can assign materials later.
            scene.add_geometry(mesh, node_name=cat)
    return scene

# --- Main Function ---
def main():
    # Configuration & Data Loading
    path_to_json = '../public/urban_district.hbjson'
    output_path = path_to_json.replace(".hbjson", ".glb")
    rotation_matrix = get_rotation_matrix(rotate=False)  # Set rotate=True if needed

    with open(path_to_json, 'r') as f:
        hbjson = json.load(f)
    rooms = hbjson.get("rooms", [])

    # Accumulate geometry per category.
    combined_results = {cat: [] for cat in SBE_COLORS.keys()}
    t0 = time.perf_counter()
    for room in rooms:
        room_geom = process_room_vectorized(room, rotation_matrix)
        for cat, items in room_geom.items():
            combined_results[cat].extend(items)
    t1 = time.perf_counter()
    print(f"Accumulated geometry from all rooms in {t1 - t0:.4f} seconds")

    # Build a scene with separate meshes per category.
    scene = build_scene_from_category_meshes(combined_results)
    t2 = time.perf_counter()
    scene.export(output_path)
    t3 = time.perf_counter()
    print(f"Exported GLB file with scene graph: {output_path}")
    print(f"Scene assembly took {t2 - t1:.4f} seconds, export took {t3 - t2:.4f} seconds")

if __name__ == "__main__":
    main()


Accumulated geometry from all rooms in 1.4361 seconds
Exported GLB file with scene graph: ../public/urban_district.gltf
Scene assembly took 0.0859 seconds, export took 0.0341 seconds
